# Geo part

In [1]:
#!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [64]:
from geopy.distance import distance
from geopy.distance import great_circle

import json, logging
from geopy.geocoders import Nominatim

def seperator(a,b):
    c = (b - a)/9
    return [round(a,7),round(a+c,7),round(a+c*2,7),round(a+c*3,7),round(a+c*4,7),round(a+c*5,7),round(a+c*6,7),round(a+c*7,7),round(a+c*8,7),round(b,7)]

def geo_point_creator(city):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(city)
    lat_list = seperator(float(location.raw['boundingbox'][0]),float(location.raw['boundingbox'][1]))
    lon_list = seperator(float(location.raw['boundingbox'][2]),float(location.raw['boundingbox'][3]))
    point_list = []
    for lat in lat_list:
        for lon in lon_list:
            point_list.append([city,lat,lon])
    df_city = pd.DataFrame(point_list, columns=['city','lat','lng'])
    point0 = [float(df_city.loc[0,['lat']]),float(df_city.loc[0,['lng']])]
    point1 = [float(df_city.loc[1,['lat']]),float(df_city.loc[1,['lng']])]
    point10 = [float(df_city.loc[10,['lat']]),float(df_city.loc[10,['lng']])]
    rad1 = int(distance(point0, point1).meters//2)
    rad2 = int(distance(point0, point10).meters//2) 
    radius = (rad1 + rad2)//2
    #print(point0,point1,point10,rad1,rad2,radius)
    df_city['radius'] = radius
    return df_city

# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import requests
import re

df = pd.read_csv('CITIES_12062020101058739.csv')

In [34]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpt_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpt_city['Flags'][i] = top5_gdpt_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x

gdpt = top5_gdpt_city['Flags'].to_list()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [35]:
top5_gdpp_city = gdp_2018[gdp_2018['VAR']=='GDP_PC_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpp_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpp_city['Flags'][i] = top5_gdpp_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x
gdpp = top5_gdpp_city['Flags'].to_list()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
CLIENT_ID = 'GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU' # your Foursquare ID
CLIENT_SECRET = 'AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU
CLIENT_SECRET:AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4


In [126]:
def getNearbyVenues(name, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        print('{},{}'.format(lat,lng))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**for loop to generate the DataFrame**

In [68]:
city_list = gdpt
city_list = ['New York, USA',
 'Los Angeles, USA',
 'Seoul, KOR',
 'Washington, USA',
 'Paris, FR']

In [ ]:
city_point = pd.DataFrame(columns=['city','lat','lng','radius'])
for city in city_list:
    df_tmp = geo_point_creator(city)
    city_point = city_point.append(df_tmp, ignore_index=True)

# Test Zone

## Task: Change the city with country code to have better geopy result (Done)

In [63]:
list1 = [['USA145','New York',''],['UK18','London','']]
df = pd.DataFrame(list1,columns=['Country','city','city_country'])
df

,Country,city,city_country
0,USA145,New York,
1,UK18,London,


In [25]:

for i,n in enumerate(top5_gdpt_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpt_city['Flags'][i] = top5_gdpt_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()


In [30]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('New York, USA')

In [31]:
print(json.dumps(location.raw,indent=4,ensure_ascii=False))

{
    "place_id": 234930245,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 175905,
    "boundingbox": [
        "40.477399",
        "40.9161785",
        "-74.25909",
        "-73.7001809"
    ],
    "lat": "40.7127281",
    "lon": "-74.0060152",
    "display_name": "New York, United States of America",
    "class": "boundary",
    "type": "administrative",
    "importance": 1.017576611451846,
    "icon": "https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png"
}


## TASK: Calculate the radius parameter(Done)

In [76]:
city = 'London'
point0 = [float(city_point[city_point['city']==city].reset_index().loc[0,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[0,['lng']])]
point1 = [float(city_point[city_point['city']==city].reset_index().loc[1,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[1,['lng']])]
point10 =  [float(city_point[city_point['city']==city].reset_index().loc[10,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[10,['lng']])]
rad1 = int(vincenty(point0, point1).meters//2)
rad2 = int(vincenty(point0, point10).meters//2)  
radius = (rad1 + rad2)//2
radius

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


1607

## TASK: GET VENUES and create feature DataFrame(Pending)

## TASK: Clustering(Pending)

In [127]:
test_venues = getNearbyVenues(name='New York',latitudes=df2['lat'],longitudes=df2['lng'])

40.477399,-74.25909
40.477399,-74.2031991
40.477399,-74.1473082
40.477399,-74.0914173
40.477399,-74.0355264


In [6]:
df = geo_point_creator('New York')

In [34]:
city_point.head(20)

,city,lat,lng
0,New York,40.477399,-74.259090
1,New York,40.477399,-74.196989
2,New York,40.477399,-74.134888
3,New York,40.477399,-74.072787
4,New York,40.477399,-74.010686
5,New York,40.477399,-73.948585
6,New York,40.477399,-73.886484
7,New York,40.477399,-73.824383
8,New York,40.477399,-73.762282
9,New York,40.477399,-73.700181


In [23]:
df

,city,lat,lng
0,New York,40.477399,-74.259090
1,New York,40.477399,-74.196989
2,New York,40.477399,-74.134888
3,New York,40.477399,-74.072787
4,New York,40.477399,-74.010686
...,...,...,...
195,London,51.667322,-0.109870
196,London,51.667322,-0.074314
197,London,51.667322,-0.038759
198,London,51.667322,-0.003203


In [71]:
lat = 53.567411	
lng = -2.474144
radius = 500
LIMIT = 100
    
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

In [72]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [56]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU&client_secret=AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4&v=20180605&ll=53.567411,-2.474144&radius=500&limit=100'

In [87]:
city_points = {}
geo_point_creator('New York')

In [51]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Oxford,UK')

In [52]:
print(json.dumps(location.raw,indent=4,ensure_ascii=False))

{
    "place_id": 235135132,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 394037,
    "boundingbox": [
        "51.7109788",
        "51.7962658",
        "-1.3042472",
        "-1.1758134"
    ],
    "lat": "51.7520131",
    "lon": "-1.2578499",
    "display_name": "Oxford, Oxfordshire, South East, England, United Kingdom",
    "class": "boundary",
    "type": "administrative",
    "importance": 0.7706040410607271,
    "icon": "https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png"
}
